## 2つの二項分布から生成される表に対するカイ2乗値が離散的であることを確認するプログラム

以下のプログラムでは，次のことを行っている．

$ Y_1 \sim Bin(n_1, \pi_1), Y_2 \sim Bin(n_2, \pi_2)$で乱数を生成し，各表に対して

- Pearsonの$X^2$検定統計量の値，および，
- mid-pを$\chi^2$分布の分位点で変換した値
 
を計算する．

複数の表に対して計算されたそれらの統計量をまとめ，ユニークな値を求める．

** 私は，Julia歴「2時間弱」なので，以下のコードはかなり汚いし，意味不明なことをしているに違いない．すみません...**



In [84]:
#バージョンの表示
versioninfo()

Julia Version 1.3.0
Commit 46ce4d7933 (2019-11-26 06:09 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i3-5005U CPU @ 2.00GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, broadwell)


In [85]:
# 確率関数のパッケージをインポート
##import Pkg; Pkg.add("Distributions")

# 確率関数のパッケージは Distributions
using Distributions

# 2×2表から，通常のPearsonカイ2乗統計量を計算する
function chisq2by2(y1, y2, n1, n2)
    # 引数は，y1 ～ Bin(n1, pi),  y2 ～ Bin(n2, p2)
    p1 = y1 / n1
    p2 = y2 / n2
    n = n1 + n2
    y = y1 + y2
    p = y / n
    if p == 0 || p == 1
        chisq = 0 # ゼロのときはカイ2乗値をゼロ
    else
        chisq = (p1 - p2)^2  / ((1/n1 + 1/n2) * p * (1-p))
    end
    return round(chisq, digits = 10) #最終的にユニークな値をカウントしたいので，ここでは丸めています
end


# 2×2表からmid-Pを計算し，そこからカイ2乗分布の分位点を計算する
function qchisq_midp_2by2(y1, y2, n1, n2)
    y = y1 + y2
    dist_hyp = Hypergeometric(n1, n2, y)
    prob_now = pdf(dist_hyp, y1)
    imax = min(n1, y)
    midp = 0    
    for i in 0:imax
        prob_temp = pdf(dist_hyp, i)
        if isapprox(prob_temp, prob_now)　# isapprox関数は，approx equalを判定
            midp += 0.5 * prob_now        # mid-p
        elseif prob_temp < prob_now      # 小さければ，そのまま足す
            midp += prob_temp
        end
    end
    chisq = quantile.(Chisq(1), 1 - midp)  # mid-pをカイ2乗分布の分位点に変換
    return round(chisq, digits = 10)       # #最終的にユニークな値をカウントしたいので，ここでは丸めています
end

nsim = 1e6 # 乱数シミュレーションの回数
n1 = 6     # 第1群の標本サイズ
n2 = 3     # 第2群の標本サイズ
pi = 1 / 3 # 二項分布でのイベントが生じる確率

dist_bin1 = Binomial(n1, pi)
dist_bin2 = Binomial(n2, pi)

chisq_pearson =  Float64[] #Juliaでの配列の使い方がよく分からないので泥臭く，空の配列を準備
chisq_midp =  Float64[]    #Juliaでの配列の使い方がよく分からないので泥臭く，空の配列を準備

for i in 1:nsim
    y1 = rand(dist_bin1)
    y2 = rand(dist_bin2)
    push!(chisq_pearson, chisq2by2(y1, y2, n1, n2))     #配列にプッシュ
    push!(chisq_midp, qchisq_midp_2by2(y1, y2, n1, n2)) #配列にプッシュ
end

# ユニークな値を取得
nlevels_pearson = unique(chisq_pearson)
nlevels_midp = unique(chisq_midp)

# 結果の表示
print("\n")
print("カイ2乗分位点", "\n")
print("通常のPearsonカイ2乗", "\n")
print(nlevels_pearson, "\n")
print(length(nlevels_pearson), "\n")
print("mid-pのカイ2乗分位点", "\n")
print(nlevels_midp, "\n")
print(length(nlevels_midp), "\n")


カイ2乗分位点
通常のPearsonカイ2乗
[0.9, 1.2857142857, 0.5625, 2.25, 0.225, 0.3214285714, 0.0, 3.6, 5.1428571429, 5.625, 9.0]
11
mid-pのカイ2乗分位点
[0.8908831485, 1.1119121515, 0.1855260064, 2.4298061542, 0.0917971519, 0.1015310443, 0.1171531678, 2.5958660697, 4.1486932801, 5.1084542765, 0.4549364231, 1.9126727556, 7.5646622112]
13
